In [2]:
# Importar librerías
import os
import pandas as pd
import numpy as np
from datetime import timedelta

import boto3
import awswrangler as wr
from botocore.config import Config

In [4]:
# Read the CSV files
orders_train = pd.read_csv('./data/clean/orders_train.csv')
orders_test = pd.read_csv('./data/clean/orders_test.csv')
orders_pred = pd.read_csv('./data/clean/orders_pred.csv')

In [9]:
# Get the max date from the orders_test DataFrame
max_date = pd.to_datetime(orders_test['date']).max()

# Create a date range for the next 30 days
future_dates = pd.date_range(start=max_date + timedelta(days=1), periods=30, freq='D')

# Assign the future dates to the orders_pred DataFrame
orders_pred['date'] = future_dates
orders_pred.rename(columns={'Predictions': 'total_sales'}, inplace=True)

# Add a 'type' column
orders_train['type'] = 'actuals'
orders_test['type'] = 'actuals'
orders_pred['type'] = 'forecast'

# Concatenate the DataFrames
all_orders_forecast = pd.concat([orders_train, orders_test, orders_pred[['date', 'total_sales', 'type']]])

# Ensure the date column is in datetime format
all_orders_forecast['date'] = pd.to_datetime(all_orders_forecast['date'])

# Sort by date
all_orders_forecast.sort_values(by='date', inplace=True)

# Reset index (optional)
all_orders_forecast.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(all_orders_forecast)

# Save to a new CSV
all_orders_forecast.to_csv('./data/AWS/all_orders_forecast.csv', index=False)


          date   total_sales      type
0   2023-11-18  52825.500000   actuals
1   2023-11-19  35925.000000   actuals
2   2023-11-20  26480.000000   actuals
3   2023-11-21   5844.000000   actuals
4   2023-11-22   9928.000000   actuals
..         ...           ...       ...
205 2024-06-10  12902.932963  forecast
206 2024-06-11  12894.596876  forecast
207 2024-06-12  12896.117998  forecast
208 2024-06-13  12903.139231  forecast
209 2024-06-14  12903.625673  forecast

[210 rows x 3 columns]


In [31]:
# Abrir cliente de S3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')
BUCKET_NAME = "itam-mgs-pf-marketplace"

In [39]:
# Upload los archivos a S3
s3.upload_file(Filename="./data/AWS/all_orders_forecast.csv", Bucket=BUCKET_NAME, Key="s../forecast/all_orders_forecast.csv")

In [34]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS marketplace.all_orders_forecast (
    date string,
    total_sales float,
    type string
    ) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
    WITH SERDEPROPERTIES ('field.delim' = ',') 
    STORED AS TEXTFILE
    LOCATION 's3://itam-mgs-pf-marketplace/s../forecast/' 
    TBLPROPERTIES ('classification' = 'csv', 'skip.header.line.count'='1');

'''

In [36]:
wr.athena.read_sql_query(
    query, 
    database="marketplace", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [38]:
query = '''
    SELECT *
    FROM marketplace.all_orders_forecast;
'''

all_orders_forecast = wr.athena.read_sql_query(
    query, 
    database="orders", 
    ctas_approach=False,
    boto3_session=session
)

all_orders_forecast.tail()

,date,total_sales,type
205,2024-06-10,12902.932617,forecast
206,2024-06-11,12894.596680,forecast
207,2024-06-12,12896.118164,forecast
208,2024-06-13,12903.139648,forecast
209,2024-06-14,12903.625977,forecast
